In [22]:
import re
import pandas as pd
from scattertext import SampleCorpora, whitespace_nlp_with_sentences
from scattertext import produce_scattertext_explorer
from scattertext.CorpusFromPandas import CorpusFromPandas
from scattertext.termscoring.ScaledFScore import ScaledFScorePresets

# regular expression to clean the tweet
RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)

#removing emojis
stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
def strip_emoji(text):
    return RE_EMOJI.sub(r'', text)

#removing hashtags, urls and mentions
def clean_tweet(tweet):
    words = []
    for element in tweet.split():
        if not (element.startswith("#") or element.startswith("http") or element.startswith("@")) :
            if element not in stopwords:
                words.append(element)
    return strip_emoji(" ".join(words))

# ==== USE THIS FUNCTIONS TO GENERATE YOUR SCATTERTEXTS =====
def generate_scatterplot_by_party(theme,right_party,left_party,right_cat,left_cat,output):
    #open the dataset
    maga = pd.read_excel("the MAGA corpus.xlsx",index_col=0)
    #selecting the features that we want to use
    maga = maga[["Tweet","Author","Theme","Bias"]]
    #======= HERE YOU CHANGE THE TOPIC YOU WANT TO USE TO GENERATE ==============
    repvsdem = maga[maga["Theme"]==theme]
    
    # processing the data
    repvsdem["Bias"] = repvsdem["Bias"].apply(lambda x: right_party if x == "right-wing" else left_party)
    repvsdem["Tweet"] = repvsdem["Tweet"].apply(clean_tweet)
    repvsdem["Author"] = repvsdem["Author"].apply(lambda x: str(x))

    
    cleaned_repvsdem = repvsdem.reset_index()[["Tweet","Author","Bias"]].rename(columns={"Tweet":"text","Author":"author","Bias":"party"})
    
    # reading the dataset with scattertext library
    # in the category parameter you enter the binary categorical feature you have chosen
    corpus = CorpusFromPandas(cleaned_repvsdem,
                              category_col='party',
                              text_col='text',
                              nlp=whitespace_nlp_with_sentences).build()
    
    #creates the scattertext plot
    html = produce_scattertext_explorer(
        corpus,
        category=left_party,
        category_name=left_cat,
        not_category_name=right_cat,
        minimum_term_frequency=5,
        pmi_threshold_coefficient=8,
        width_in_pixels=1000,
        metadata=cleaned_repvsdem['author'],
        #term_scorer=ScaledFScorePresets(one_to_neg_one=True, beta=1),
        d3_scale_chromatic_url='https://cdn.jsdelivr.net/npm/d3-scale-chromatic@1.3.3/dist/d3-scale-chromatic.min.js',
        d3_url='https://cdnjs.cloudflare.com/ajax/libs/d3/4.6.0/d3.min.js',
    )

    open(output, 'wb').write(html.encode('utf-8'))
    print('Open %s in Chrome or Firefox.'%output)

In [3]:
generate_scatterplot_by_party("Republicans vs Democrats","republican","democrat",'Republican','Democratic','../results/scattertexts/repvsdem.html')

Open ../results/scattertexts/repvsdem.html in Chrome or Firefox.


In [4]:
generate_scatterplot_by_party("Tories vs Labour","tories","democrat",'Tories','Labour','../results/scattertexts/torvslab.html')

Open ../results/scattertexts/torvslab.html in Chrome or Firefox.


In [6]:
generate_scatterplot_by_party("Brexit","right","left",'Right-wing','Left-wing','../results/scattertexts/brexit.html')

Open ../results/scattertexts/brexit.html in Chrome or Firefox.


In [7]:
generate_scatterplot_by_party("New Zealand politics","right","left",'Right-wing','Left-wing','../results/scattertexts/new-zealand.html')

Open ../results/scattertexts/new-zealand.html in Chrome or Firefox.


In [25]:
generate_scatterplot_by_party("Brexit","right","left",'Right-wing','Left-wing','../results/scattertexts/canada.html')

Open ../results/scattertexts/canada.html in Chrome or Firefox.
